In [106]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib

In [107]:
from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/data_movies.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [108]:
data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [109]:
data.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [110]:
features = ['genres','keywords','tagline','cast','director','crew','title']

for feature in features:
  data[feature] = data[feature].fillna('')

In [111]:
data['combine'] = data['genres']+' '+data['keywords']+' '+data['tagline']+' '+data['cast']+' '+data['director']+' '+data['crew']+' '+data['title']
data['combine'].head()

,combine
0,Action Adventure Fantasy Science Fiction cultu...
1,Adventure Fantasy Action ocean drug abuse exot...
2,Action Adventure Crime spy based on novel secr...
3,Action Crime Drama Thriller dc comics crime fi...
4,Action Adventure Science Fiction based on nove...


In [112]:
vector = TfidfVectorizer()
f_vector = vector.fit_transform(data['combine'])

In [113]:
similarity = cosine_similarity(f_vector)
print(similarity[0][5])

0.627282294123594


In [114]:
every_title = data['title'].tolist()
print(every_title[5])
print("Total movies:", len(every_title))

Spider-Man 3
Total movies: 4803


In [115]:
movie_name = input('Enter your movie name : ')

Enter your movie name : Spider Man


In [116]:
find_close_match = difflib.get_close_matches(movie_name, every_title)
print(find_close_match)

['Spider-Man', 'Spider-Man 3', 'Spider-Man 2']


In [117]:
closest_match = find_close_match[0]
print(closest_match)

Spider-Man


In [118]:
index_of_movie = data[data.title == closest_match]['index'].values[0]
print(index_of_movie)

159


In [119]:
data['title'][index_of_movie]

'Spider-Man'

In [120]:
similar_score = list(enumerate(similarity[index_of_movie]))
print(similar_score[:5])

[(0, np.float64(0.7752338694243776)), (1, np.float64(0.5963068703006155)), (2, np.float64(0.783375187233388)), (3, np.float64(0.8165944773019609)), (4, np.float64(0.7699035153333552))]


In [121]:
similar_score_sort = sorted(similar_score, key = lambda x:x[1], reverse = True)
print(similar_score_sort[:5])

[(159, np.float64(1.0000000000000007)), (28, np.float64(0.8285163494728113)), (412, np.float64(0.8239802830983093)), (212, np.float64(0.8218694075974545)), (30, np.float64(0.8192631785722463))]


In [122]:
for i in range(30):
  index = similar_score_sort[i][0]
  print(data['title'][index])

Spider-Man
Jurassic World
The Core
The Day After Tomorrow
Spider-Man 2
War of the Worlds
Contact
The Dark Knight Rises
Bad Boys II
Master and Commander: The Far Side of the World
Cursed
Men in Black II
Catwoman
Minority Report
The Wolf of Wall Street
Hulk
Blade: Trinity
Tears of the Sun
Terminator 3: Rise of the Machines
The League of Extraordinary Gentlemen
Alexander
Freddy vs. Jason
Big Fish
xXx
The Aviator
Blade II
Blade
End of Days
A.I. Artificial Intelligence
The Bourne Supremacy


In [123]:
joblib.dump(data, 'data.pkl')
joblib.dump(similarity, 'similarity.pkl')
joblib.dump(vector, 'tfidf.pkl')

['tfidf.pkl']